**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_chipseq_tf_subset")
fnames = dir(fdiry)
print(length(fnames))
print(head(fnames))

[1] 16
[1] "peak.annotation.K562.ENCSR000BMH.ENCFF398UQZ.YY1.bed.gz"  
[2] "peak.annotation.K562.ENCSR000BMW.ENCFF707MDI.REST.bed.gz" 
[3] "peak.annotation.K562.ENCSR000EFS.ENCFF190CGV.JUN.bed.gz"  
[4] "peak.annotation.K562.ENCSR000EFT.ENCFF657CTC.GATA1.bed.gz"
[5] "peak.annotation.K562.ENCSR000EFU.ENCFF715WGN.ELK1.bed.gz" 
[6] "peak.annotation.K562.ENCSR000EGE.ENCFF702XPO.EP300.bed.gz"


In [3]:
lst = str_split(string = fnames, pattern = "\\.")
for (vec in head(lst)){
    index  = vec[5]
    target = vec[6]
    label  = paste(target, index, sep="_")
    cat(vec,   "\n")
    cat(label, "\n")
    cat("\n")
}

peak annotation K562 ENCSR000BMH ENCFF398UQZ YY1 bed gz 
YY1_ENCFF398UQZ 

peak annotation K562 ENCSR000BMW ENCFF707MDI REST bed gz 
REST_ENCFF707MDI 

peak annotation K562 ENCSR000EFS ENCFF190CGV JUN bed gz 
JUN_ENCFF190CGV 

peak annotation K562 ENCSR000EFT ENCFF657CTC GATA1 bed gz 
GATA1_ENCFF657CTC 

peak annotation K562 ENCSR000EFU ENCFF715WGN ELK1 bed gz 
ELK1_ENCFF715WGN 

peak annotation K562 ENCSR000EGE ENCFF702XPO EP300 bed gz 
EP300_ENCFF702XPO 



## Import data

In [4]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_chipseq_tf_subset")
fname = "description.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath)

dat_cnames = dat
print(dim(dat))
dat

Rows: 10 Columns: 2
── Column specification ─────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): Name, Description

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 10  2


Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start position
End,End position
Name,Name given to a region
Score,"Scaled IDR value, min(int(log2(-125IDR), 1000)."
Strand,[+-.]; Use '.' if no strand is assigned.
SignalValue,Measurement of enrichment for the region for merged peaks.
PValue,Measurement of statistical significance (-log10). Use -1 if no pValue is assigned.
QValue,Measurement of statistical significance using false discovery rate (-log10). Use -1 if no qValue is assigned.


In [5]:
annotation = "ChIPseq_TF"
cnames = dat_cnames$Name
cnames = c("Chrom_ATAC", "Start_ATAC", "End_ATAC", cnames, "Overlap")

fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_chipseq_tf_subset")
fnames = dir(fdiry)

lst = str_split(string = fnames, pattern = "\\.")
lst = lapply(lst, function(vec){
    
    ### parse info
    index  = vec[5]
    target = vec[6]
    label  = paste(target, index, sep="_")
    
    ### show progress
    cat("Read:", label, "\n"); flush.console()
    
    ### set file path
    fname = paste("*", index, target, "bed.gz", sep = ".")
    fglob = file.path(fdiry, fname)
    fpath = Sys.glob(fglob)
    
    ### read data
    dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
    dat = dat %>% dplyr::mutate(
        Peak_ATAC  = paste(Chrom_ATAC, Start_ATAC, End_ATAC, sep="_"),
        #Peak_ATAC  = paste0(Chrom_ATAC, ":", Start_ATAC, "_", End_ATAC),
        Annotation = annotation,
        Label      = label
    )
    return(dat)
})

lst_peak_annot = lst
print(length(lst))
head(lst[[1]])

Read: YY1_ENCFF398UQZ 
Read: REST_ENCFF707MDI 
Read: JUN_ENCFF190CGV 
Read: GATA1_ENCFF657CTC 
Read: ELK1_ENCFF715WGN 
Read: EP300_ENCFF702XPO 
Read: CTCF_ENCFF660GHM 
Read: TAL1_ENCFF852ZRK 
Read: ZEB2_ENCFF242AOL 
Read: ATF1_ENCFF627RSK 
Read: KLF10_ENCFF142ZTD 
Read: POLR2A_ENCFF355MNE 
Read: NFE2_ENCFF023IFO 
Read: KLF16_ENCFF488OTN 
Read: SP1_ENCFF553GPK 
Read: NRF1_ENCFF777PKJ 
[1] 16


Chrom_ATAC,Start_ATAC,End_ATAC,Chrom,Start,End,Name,Score,Strand,SignalValue,PValue,QValue,Peak,Overlap,Peak_ATAC,Annotation,Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
chr1,777949,779437,chr1,778486,778776,.,1000,.,132.48683,-1,4.33080,136,290,chr1_777949_779437,ChIPseq_TF,YY1_ENCFF398UQZ
chr1,903633,905556,chr1,904638,904883,.,1000,.,75.95987,-1,4.33080,112,245,chr1_903633_905556,ChIPseq_TF,YY1_ENCFF398UQZ
chr1,912650,914455,chr1,913032,913129,.,1000,.,41.88374,-1,3.88398,56,97,chr1_912650_914455,ChIPseq_TF,YY1_ENCFF398UQZ
chr1,920697,921526,chr1,921136,921238,.,1000,.,50.33578,-1,4.33080,62,102,chr1_920697_921526,ChIPseq_TF,YY1_ENCFF398UQZ
chr1,923417,925194,chr1,923707,924017,.,1000,.,34.69952,-1,3.76640,155,310,chr1_923417_925194,ChIPseq_TF,YY1_ENCFF398UQZ
chr1,923417,925194,chr1,924711,925021,.,634,.,20.27464,-1,3.58222,155,310,chr1_923417_925194,ChIPseq_TF,YY1_ENCFF398UQZ


## Arrange and summarize

In [6]:
lst = lst_peak_annot

lst = lapply(lst, function(dat){
    dat = dat %>% 
        dplyr::group_by(Peak_ATAC, Annotation, Label) %>%
        dplyr::summarise(Count = n(), .groups = "drop")
    return(dat)
})

dat = bind_rows(lst)
dat = dat %>% dplyr::rename("Peak" = "Peak_ATAC")

dat_peak_annot_count = dat
print(dim(dat))
head(dat)

[1] 270491      4


Peak,Annotation,Label,Count
<chr>,<chr>,<chr>,<int>
chr10_100009096_100010466,ChIPseq_TF,YY1_ENCFF398UQZ,1
chr10_100185017_100187275,ChIPseq_TF,YY1_ENCFF398UQZ,1
chr10_100228452_100230090,ChIPseq_TF,YY1_ENCFF398UQZ,1
chr10_100267066_100268374,ChIPseq_TF,YY1_ENCFF398UQZ,1
chr10_100285974_100287341,ChIPseq_TF,YY1_ENCFF398UQZ,2
chr10_100346328_100347693,ChIPseq_TF,YY1_ENCFF398UQZ,1


In [7]:
fdiry = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input", "summary")
fname = "peak.summary.chipseq_tf_subset.tsv"
fpath = file.path(fdiry, fname)

dat = dat_peak_annot_count
write_tsv(dat, fpath)